In [32]:
import pandas as pd

In [33]:
df = pd.read_csv('income(1) (1).csv')
df

,age,JobType,EdType,maritalstatus,occupation,relationship,race,gender,capitalgain,capitalloss,hoursperweek,nativecountry,SalStat
0,45,Private,HS-grad,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,28,United-States,"less than or equal to 50,000"
1,24,Federal-gov,HS-grad,Never-married,Armed-Forces,Own-child,White,Male,0,0,40,United-States,"less than or equal to 50,000"
2,44,Private,Some-college,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,"greater than 50,000"
3,27,Private,9th,Never-married,Craft-repair,Other-relative,White,Male,0,0,40,Mexico,"less than or equal to 50,000"
4,20,Private,Some-college,Never-married,Sales,Not-in-family,White,Male,0,0,35,United-States,"less than or equal to 50,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31973,34,Local-gov,HS-grad,Never-married,Farming-fishing,Not-in-family,Black,Male,594,0,60,United-States,"less than or equal to 50,000"
31974,34,Local-gov,Some-college,Never-married,Protective-serv,Not-in-family,White,Female,0,0,40,United-States,"less than or equal to 50,000"
31975,23,Private,Some-college,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States,"less than or equal to 50,000"
31976,42,Local-gov,Some-college,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,40,United-States,"less than or equal to 50,000"


In [34]:
df.shape

(31978, 13)

In [35]:
from sklearn.preprocessing import LabelEncoder
df_encoded = df.copy()
le = LabelEncoder()
for col in df_encoded.select_dtypes(include='object').columns:
    df_encoded[col] = le.fit_transform(df_encoded[col])
X = df_encoded.drop('SalStat', axis=1)
y = df_encoded['SalStat']

In [36]:
print(X)

       age  JobType  EdType  maritalstatus  occupation  relationship  race  \
0       45        4      11              0           1             1     4   
1       24        1      11              4           2             3     4   
2       44        4      15              2          10             0     4   
3       27        4       6              4           3             2     4   
4       20        4      15              4          12             1     4   
...    ...      ...     ...            ...         ...           ...   ...   
31973   34        2      11              4           5             1     2   
31974   34        2      15              4          11             1     4   
31975   23        4      15              2           1             0     4   
31976   42        2      15              2           1             5     4   
31977   29        4       9              4          10             1     4   

       gender  capitalgain  capitalloss  hoursperweek  nativeco

In [37]:
print(y)

0        1
1        1
2        0
3        1
4        1
        ..
31973    1
31974    1
31975    1
31976    1
31977    1
Name: SalStat, Length: 31978, dtype: int64


In [38]:
from sklearn.feature_selection import SelectKBest, chi2
chi2_selector = SelectKBest(chi2, k=5)
chi2_selector.fit(X, y)

selected_features_chi2 = X.columns[chi2_selector.get_support()]
print("Selected Features (Chi-Square):", list(selected_features_chi2))

mi_score = pd.Series(chi2_selector.scores_, index=X.columns)
print(mi_score.sort_values(ascending = False))

Selected Features (Chi-Square): ['age', 'relationship', 'capitalgain', 'capitalloss', 'hoursperweek']
capitalgain      7.937393e+07
capitalloss      1.326247e+06
age              8.442446e+03
hoursperweek     6.403432e+03
relationship     3.588405e+03
maritalstatus    1.098466e+03
occupation       5.163211e+02
gender           4.952462e+02
EdType           2.985354e+02
JobType          4.599636e+01
race             3.244850e+01
nativecountry    1.761006e+01
dtype: float64


In [39]:
from sklearn.feature_selection import f_classif
anova_selector = SelectKBest(f_classif, k=5)
anova_selector.fit(X, y)

selected_features_anova = X.columns[anova_selector.get_support()]
print("Selected Features (ANOVA F-test):", list(selected_features_anova))

mi_score = pd.Series(anova_selector.scores_, index=X.columns)
print(mi_score.sort_values(ascending = False))

Selected Features (ANOVA F-test): ['age', 'relationship', 'gender', 'capitalgain', 'hoursperweek']
relationship     2147.529577
age              1845.619922
hoursperweek     1793.365964
capitalgain      1668.654212
gender           1565.942812
maritalstatus    1311.153437
capitalloss       729.540003
EdType            206.255905
occupation        190.369850
race              169.960117
JobType            83.931000
nativecountry      17.189315
dtype: float64


In [40]:
from sklearn.feature_selection import mutual_info_classif
mi_selector = SelectKBest(mutual_info_classif, k=5)
mi_selector.fit(X, y)

selected_features_mi = X.columns[mi_selector.get_support()]
print("Selected Features (Mutual Info):", list(selected_features_mi))

mi_score = pd.Series(mi_selector.scores_, index=X.columns)
print("\nMutual Information Scores:\n", mi_score.sort_values(ascending = False))

Selected Features (Mutual Info): ['age', 'maritalstatus', 'occupation', 'relationship', 'capitalgain']

Mutual Information Scores:
 relationship     0.114306
maritalstatus    0.108172
capitalgain      0.083520
age              0.072000
occupation       0.063377
EdType           0.061627
hoursperweek     0.043028
capitalloss      0.035169
gender           0.029758
JobType          0.016155
race             0.013426
nativecountry    0.008322
dtype: float64


In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

model = LogisticRegression(max_iter=100)
rfe = RFE(model, n_features_to_select=5)
rfe.fit(X, y)

selected_features_rfe = X.columns[rfe.support_]
print("Selected Features (RFE):", list(selected_features_rfe))

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Selected Features (RFE): ['EdType', 'maritalstatus', 'relationship', 'race', 'gender']


In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SequentialFeatureSelector as SFS

model = LogisticRegression(max_iter=100)
sfs = SFS(model, n_features_to_select=5)
sfs.fit(X, y)

selected_features_sfs = X.columns[sfs.support_]
print("Selected Features (SFS):", list(selected_features_sfs))

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Selected Features (SFS): ['JobType', 'maritalstatus', 'capitalgain', 'capitalloss', 'nativecountry']


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [43]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X, y)
importances = pd.Series(rf.feature_importances_, index=X.columns)
selected_features_rf = importances.sort_values(ascending=False).head(5).index
print("Top 5 features (Random Forest Importance):", list(selected_features_rf))

Top 5 features (Random Forest Importance): ['age', 'capitalgain', 'hoursperweek', 'relationship', 'EdType']


In [44]:
from sklearn.linear_model import LassoCV
import numpy as np

lasso = LassoCV(cv=5).fit(X, y)
importance = np.abs(lasso.coef_)

selected_features_lasso = X.columns[importance > 0]
print("Selected Features (Lasso):", list(selected_features_lasso))

Selected Features (Lasso): ['age', 'capitalgain', 'capitalloss', 'hoursperweek']


In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
df = pd.read_csv('income(1) (1).csv')
le = LabelEncoder()
df_encoded = df.copy()
for col in df_encoded.select_dtypes(include='object').columns:
    df_encoded[col] = le.fit_transform(df_encoded[col])
X = df_encoded.drop('SalStat', axis=1)
y = df_encoded['SalStat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8198874296435272

Confusion Matrix:
 [[1350  976]
 [ 752 6516]]

Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.58      0.61      2326
           1       0.87      0.90      0.88      7268

    accuracy                           0.82      9594
   macro avg       0.76      0.74      0.75      9594
weighted avg       0.81      0.82      0.82      9594

